### Baixar as DFPs (2019–2023) da Embraer em XBRL diretamente do site da CVM

In [2]:
# bibliotecas importadas
import requests
import zipfile
import os
import pandas as pd
import io

In [4]:
# ===== Configurações =====
ANOS = range(2020, 2025)  # 2020, 2021, 2022, 2023, 2024
URL_BASE = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_{}.zip"
EMPRESA = "EMBRAER S.A."
PASTA_ZIP = "zips"
PASTA_SAIDA = "saida"

# Nome-padrão dentro do zip para os demonstrativos consolidados
DEMONSTRATIVOS = {
    "Ativo": "BPA_con",
    "Passivo": "BPP_con",
    "DRE": "DRE_con",
    "DFC": "DFC_MI_con"
}

# Criar pastas se não existirem
os.makedirs(PASTA_ZIP, exist_ok=True)
os.makedirs(PASTA_SAIDA, exist_ok=True)

# Dicionário para acumular DataFrames por demonstração
dfs = {nome: [] for nome in DEMONSTRATIVOS.keys()}

for ano in ANOS:
    print(f"🔽 Baixando ano {ano}...")
    url = URL_BASE.format(ano)
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"❌ Não conseguiu baixar para o ano {ano}. Status: {resp.status_code}")
        continue

    # Salvar zip localmente opcionalmente
    caminho_zip = os.path.join(PASTA_ZIP, f"dfp_cia_aberta_{ano}.zip")
    with open(caminho_zip, "wb") as f:
        f.write(resp.content)
    print(f"✅ Zip salvo: {caminho_zip}")

    # Abrir zip em memória
    with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
        nomes_zip = z.namelist()
        for nome_dem, padrao in DEMONSTRATIVOS.items():
            # encontrar arquivo CSV consolidado que contenha o padrão
            # ex: “dfp_cia_aberta_BPA_con_2023.csv”
            possiveis = [f for f in nomes_zip if padrao in f and f.endswith(".csv")]
            if not possiveis:
                print(f"⚠️ Não achei arquivo {padrao} para ano {ano}")
                continue

            # Escolher o primeiro que encontrar
            nome_arq = possiveis[0]
            print(f"   ↳ Extraindo {nome_arq} para {nome_dem}, ano {ano}")

            with z.open(nome_arq) as arq:
                # Ler CSV com encoding e separador certos; pode ajustar dependendo do CSV
                df = pd.read_csv(arq, sep=";", encoding="latin1", dtype=str)

            # Filtrar para Embraer
            df_emb = df[df["DENOM_CIA"].str.strip().str.upper() == EMPRESA.upper()]
            if df_emb.empty:
                print(f"      ‼️ Nenhum registro para Embraer no {nome_dem}, ano {ano}")
            else:
                # Adicionar coluna ano para pivotar depois ou para checagem
                df_emb["ANO"] = ano
                dfs[nome_dem].append(df_emb)

# Após o loop, concatenar e salvar CSVs
for nome_dem, lista in dfs.items():
    if not lista:
        print(f"⚠️ Sem dados para {nome_dem} nos anos especificados.")
        continue
    df_total = pd.concat(lista, ignore_index=True)
    caminho_saida = os.path.join(PASTA_SAIDA, f"{nome_dem}_Embraer_2020-2024.csv")
    df_total.to_csv(caminho_saida, index=False, encoding="utf-8-sig", sep=";")
    print(f"✅ Salvou {caminho_saida}")


🔽 Baixando ano 2020...
✅ Zip salvo: zips\dfp_cia_aberta_2020.zip
   ↳ Extraindo dfp_cia_aberta_BPA_con_2020.csv para Ativo, ano 2020


C:\Users\Maria Eduarda\AppData\Local\Temp\ipykernel_6668\59865357.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = ano


   ↳ Extraindo dfp_cia_aberta_BPP_con_2020.csv para Passivo, ano 2020


C:\Users\Maria Eduarda\AppData\Local\Temp\ipykernel_6668\59865357.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = ano


   ↳ Extraindo dfp_cia_aberta_DRE_con_2020.csv para DRE, ano 2020


...

   ↳ Extraindo dfp_cia_aberta_DRE_con_2024.csv para DRE, ano 2024


C:\Users\Maria Eduarda\AppData\Local\Temp\ipykernel_6668\59865357.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = ano


   ↳ Extraindo dfp_cia_aberta_DFC_MI_con_2024.csv para DFC, ano 2024


C:\Users\Maria Eduarda\AppData\Local\Temp\ipykernel_6668\59865357.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb["ANO"] = ano


✅ Salvou saida\Ativo_Embraer_2020-2024.csv
✅ Salvou saida\Passivo_Embraer_2020-2024.csv
✅ Salvou saida\DRE_Embraer_2020-2024.csv
✅ Salvou saida\DFC_Embraer_2020-2024.csv


In [5]:
for nome_dem in DEMONSTRATIVOS.keys():
    caminho = os.path.join(PASTA_SAIDA, f"{nome_dem}_Embraer_2020-2024.csv")
    if not os.path.exists(caminho):
        continue
    df = pd.read_csv(caminho, sep=";", encoding="utf-8-sig", dtype=str)
    # Selecionar colunas necessárias
    # Exemplo típico:
    df_sel = df[["CD_CONTA", "DS_CONTA", "ANO", "VL_CONTA"]]
    # Pivot
    df_pivot = df_sel.pivot_table(index=["CD_CONTA", "DS_CONTA"], 
                                  columns="ANO", values="VL_CONTA", aggfunc="first").reset_index()
    # Salvar no lugar ou outro nome
    novo = os.path.join(PASTA_SAIDA, f"{nome_dem}_contabil_2020-2024.csv")
    df_pivot.to_csv(novo, index=False, encoding="utf-8-sig", sep=";")
    print(f"📦 Salvou versão contabil: {novo}")

📦 Salvou versão contabil: saida\Ativo_contabil_2020-2024.csv
📦 Salvou versão contabil: saida\Passivo_contabil_2020-2024.csv
📦 Salvou versão contabil: saida\DRE_contabil_2020-2024.csv
📦 Salvou versão contabil: saida\DFC_contabil_2020-2024.csv


In [7]:
ARQUIVO_XLSM = "Demonstrações_Embraer.xlsm"

# Arquivos de entrada
arquivos = {
    "Ativo": "Ativo_Embraer_2020-2024.csv",
    "Passivo": "Passivo_Embraer_2020-2024.csv",
    "DRE": "DRE_Embraer_2020-2024.csv",
    "DFC": "DFC_Embraer_2020-2024.csv"  # pode trocar para DFC_MD se quiser
}

with pd.ExcelWriter(ARQUIVO_XLSM, engine="openpyxl") as writer:
    for aba, nome_arq in arquivos.items():
        caminho = os.path.join(PASTA_SAIDA, nome_arq)
        if not os.path.exists(caminho):
            print(f"⚠️ Arquivo não encontrado: {caminho}")
            continue

        # Ler CSV
        df = pd.read_csv(caminho, sep=";", encoding="utf-8-sig")

        # Pivotar: Conta x Ano
        if "CD_CONTA" in df.columns and "DS_CONTA" in df.columns:
            df["Conta"] = df["CD_CONTA"] + " - " + df["DS_CONTA"]
        else:
            df["Conta"] = df["DS_CONTA"]

        # Converter valores para numérico (quando possível)
        df["VALOR"] = pd.to_numeric(df["VL_CONTA"], errors="coerce")

        tabela = df.pivot_table(
            index="Conta",
            columns="ANO",
            values="VALOR",
            aggfunc="first"
        ).reset_index()

        # Salvar na aba correspondente
        tabela.to_excel(writer, sheet_name=aba, index=False)

print(f"✅ Arquivo Excel criado: {ARQUIVO_XLSM}")

✅ Arquivo Excel criado: Demonstrações_Embraer.xlsm
